__DELETE ME__
### code flow 
- api 한 번 호출할때마다 2000자까지 가능이라 리뷰 50개씩 40번, 총 상위 2000개의 리뷰를 처리함 (리뷰 개수 늘려도 됩니당)
- 1루프마다 (50개씩 돌릴때마다)
    - 50개의 리뷰를 한 문장으로 이어붙이고, 이걸 api request로 보냄
    - 루프마다 리뷰 요약 안되면 에러뜨고 넘어감
    - 요약 결과 잘 넘어오면 response_list에 저장함
- 루프 다 끝나면 최종 요약 생성
    - response_list의 값들을 한 문장으로 이어붙이고, 이걸 또 api request로 보냄
    - 최종 요약 결과 잘 넘어오면 return

In [26]:
# read data
def read_data(df):
    # the api can only handle 2000 characters at a time
    # so we will split the reviews into 40 chunks of 50 reviews each
    reviews_for_loop = []

    for i in range(1, 41):
        start_index = (i - 1) * 50
        end_index = i * 50
        selected_rows = df.iloc[start_index:end_index]
        
        reviews_list = selected_rows['review'].tolist()
        reviews_concatenated = ''.join(reviews_list)
        
        reviews_for_loop.append(reviews_concatenated)
    return reviews_for_loop


In [33]:
import os
import json
import requests

# 1. get the summary of each chunk
# 2. append the summary to a list
# 3. join the list into one string
# 4. get the final summary of the joined string
response_list = []
def generate_summary(i, count , reviews):
    headers = {
        "X-NCP-APIGW-API-KEY-ID": os.environ.get('client_id'),
        "X-NCP-APIGW-API-KEY": os.environ.get('client_secret'),
        "Content-Type": "application/json"
    }
    language = "ko"
    model = "general"
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = count # number of sentences for the summarized document.
    # DELETE ME: 2번의 summary 할때는 1줄씩, 4번의 final summary 할때는 2줄 출력
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= "Review Summary of 'The Outlaws'"
    content = reviews
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        response_list.append(response.text[12:-2])
        if ( i != "final") :
            print (i, "summary: ", response.text[12:-2])

In [35]:
## main function

import pandas as pd

file_path = "../result/result.csv"
df = pd.read_csv(file_path)

data = read_data(df)
for i in range(len(data)): 
    generate_summary(i+1, 1, data[i]) # set summaryCount to 1

1 summary:  한국영화의 퇴보 상업성만따지고 작품성은버림 이런 내용도없는 잔인한 영화는 이세상에서 사라져야합니다.
2 summary:  살짝어색한마동석이엇지만 전체내용은 재밋음 그럭저럭 킬링타임용으로 괜찮음..ps1)마동석만 너무 쎈캐임..ps2)윤계상 못생김 주의.... ksd평점으로 7점 매깁니다.
4 summary:  연기를허벌잘해서 개꾸르꾸르핵토파스칼잼 스트레스잼 졷선족잼 어딘가 좀 엉성하게 전체적인 짜임새가 부족하다 쓸데없는 장난이 너무들어감 노잼 별로 재미없는것 같습니다.
5 summary:  마동석 졸잼 윤계상 연기굿 지나치게 과대평가된듯 해서 평점 좀 깎습니다.
6 summary:  참 그놈의 상업을 위한, 흥행을 위한 요소들만 결부시키는 영화들만 만들어 대니 한국 영화의 발전이 없는것입니다.
7 summary:  정말 더럽게 잔인하고 화나는 영화였습니다.
8 summary:  재미 이렇게 흥행해서 감독이 한국영화판 쉽게보고 또 이딴 영화만들어 말아먹고 은퇴하고..20년전이란 변한게 없는 장르물 마동석 나오는 즉시 모든 긴장감이 사라집니다.
10 summary:  2시간 내내 보기 불편 이딴 영화를 만들었냐 진짜 보는내내 두시간이 어떻게 지나가는지 모르는영화였습니다.
11 summary:  마동석 짱 윤계상 연기잘함 저는 잔인하지만 너무 몰입도있게 봤습니다.
12 summary:  윤계상 마동석 환상의 연기 콤비 영화 많이 보진 않지만, 올해 최고 흥행인 택시운전사보다 1.5배 재밌게 봤습니다.
13 summary:  이번 년도 본 영화 중에 최고였습니다.
14 summary:  이영화 보니까 황해가 생각나서 다시 보려고 합니다.
15 summary:  잔인한 오락영화지만 시원하기도 한 느낌.강추 근래 본 영화중 최고입니다.
16 summary:  별삭감 올해 본 영화 중 5손가락 안에 듭니다.
17 summary:  잔인하고 요즘 범죄영화에서 꼭 나오는 망치질 칼 난도질.. 눈살 찌푸려집니다.
18 summary:  영화관에서 지루하지 

In [37]:
# get the final summary

final_summarize = ''.join(response_list)
summary = generate_summary("final", 2, final_summarize) # set summaryCount to 2

print("-----------final summary-----------")
print(response_list[-1]) # final summary

-----------final summary-----------
윤계상 마동석 환상의 연기 콤비 영화 많이 보진 않지만, 올해 최고 흥행인 택시운전사보다 1.5배 재밌게 봤습니다.\n최근본영화중에 최고 존잼 킬링타임용영화 인듯 마동석 윤계상 배우들에게는 최고의 영화라고 자부할 수있겠습니다.
